<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [3]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [4]:
connection = psycopg2.connect(
    dbname='project_sql',
    user='skillfactory',
    host='84.201.134.129',
    password='cCkxxLVrDE8EbvjueeMedPKt',
    port=5432
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
query_3_1 = f'''select 
                    count(*) as vacancies_count
                from 
                    public.vacancies
'''

In [6]:
pd.read_sql_query(query_3_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\1467523473.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_1, connection)


,vacancies_count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
query_3_2 = f'''select 
                    count(*) as employers_count
                from 
                    public.employers
'''

In [8]:
pd.read_sql_query(query_3_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\1321910889.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_2, connection)


,employers_count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
query_3_3 = f'''select 
                    count(*) as areas_count
                from 
                    public.areas
'''

In [10]:
pd.read_sql_query(query_3_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\169222152.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_3, connection)


,areas_count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
query_3_4 = f'''select 
                    count(*) as industries_count
                from 
                    public.industries
'''

In [12]:
pd.read_sql_query(query_3_4, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\2934468070.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_4, connection)


,industries_count
0,294


***

In [13]:
#Составлен запрос, который показывает, какое количество работодателей, регионов и сфер деятельности
#соотносят с вакансиями, имеющимися в базе данных. Запрос позволит понять,
#какая доля работодателей, регионов и сфер деятельности не связаны с имеющимися вакансиями.
query_3_add_1 = f'''select 
                        count(distinct a.id) as areas_with_vacancies,
                        count(distinct e.id) as employers_with_vacancies,
                        count(distinct ei.industry_id) as industries_with_vacancies
                    from 
                        public.vacancies as v
                    join 
                        public.areas as a on v.area_id = a.id
                    join
                        public.employers as e on v.employer_id = e.id
                    join
                        public.employers_industries as ei on e.id = ei.employer_id
'''

In [14]:
pd.read_sql_query(query_3_add_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\545387563.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_3_add_1, connection)


,areas_with_vacancies,employers_with_vacancies,industries_with_vacancies
0,711,9947,293


In [12]:
#В нашем распоряжении база данных, которая содержит в общей сложности 49 197 вакансий. 
#В нашей базе данных также есть информация относительно 23 501 потенциальном работодателе, 
#1 362 регионе и 294 сферах деятельности. При этом вакансии в базе данных представлены только
#в отношении 9 947 работодателей и 771 регионов. Вакансии имеются во всех сферах деятельности за исключением одной.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [15]:
#Для формирования запроса объединены таблицы с данными о вакансиях и регионах.
#Данные сгруиированы по названию регионов.
n = 5
query_4_1 = f'''select 
                    a.name as area,
                    count(v.id) as vacancies_count
                from 
                    public.vacancies as v
                join 
                    public.areas as a on v.area_id = a.id
                group by 
                    a.name
                order by 
                    count(v.id) desc
                limit {n}
'''

In [16]:
pd.read_sql_query(query_4_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\1702194938.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_1, connection)


,area,vacancies_count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [17]:
#Для формирования запроса данные отфильтрованы по наличию сведений о зарплате в описании вакансий.
query_4_2 = f'''select 
                    count(*) as vacancies_with_salary
                from 
                    public.vacancies
                where
                    salary_from is not null 
                    or salary_to is not null      
'''

In [18]:
pd.read_sql_query(query_4_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\495373726.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_2, connection)


,vacancies_with_salary
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [19]:
query_4_3 = f'''select 
                    round(avg(salary_from)) as avg_salary_min,
                    round(avg(salary_to)) as avg_salary_max
                from 
                    public.vacancies    
'''

In [20]:
# результат запроса
pd.read_sql_query(query_4_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\3534961570.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_3, connection)


,avg_salary_min,avg_salary_max
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [21]:
#Для формирования запроса данные сгруппированы по типу рабочего графика и типу трудойстройства.
query_4_4 = f'''select 
                    schedule,
                    employment,
                    count(*) as vacancies_count
                from 
                    public.vacancies
                group by
                    schedule, employment
                order by count(*) desc
'''

In [22]:
pd.read_sql_query(query_4_4, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\3130347181.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_4, connection)


,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [23]:
#Для формирования запроса данные сгруппированы по требуемому опыту работу.
query_4_5 = f'''select 
                    experience,
                    count(*) as experience_count
                from 
                    public.vacancies
                group by
                    experience
                order by count(*) desc
'''

In [24]:
pd.read_sql_query(query_4_5, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\3496489661.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_5, connection)


,experience,experience_count
0,От 1 года до 3 лет,26152
1,От 3 до 6 лет,14511
2,Нет опыта,7197
3,Более 6 лет,1337


***

In [25]:
#Составлен запрос, позволяющий увидеть, как различаются среднее значение
#нижней и верхней границы в зависимости от региона размещения вакансии.
n = 10
query_4_add_1 = f'''select 
                    a.name as area,
                    round(avg(salary_from)) as avg_salary_min,
                    round(avg(salary_to)) as avg_salary_max
                from 
                    public.vacancies as v
                join 
                    public.areas as a on v.area_id = a.id
                group by 
                    a.name
                order by 
                    count(v.id) desc
                limit {n}
'''

In [26]:
pd.read_sql_query(query_4_add_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\3817656583.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_add_1, connection)


,area,avg_salary_min,avg_salary_max
0,Москва,113409.0,160594.0
1,Санкт-Петербург,89892.0,127249.0
2,Минск,68040.0,133797.0
3,Новосибирск,91272.0,137573.0
4,Алматы,53498.0,96238.0
5,Екатеринбург,77489.0,120404.0
6,Нижний Новгород,83734.0,126921.0
7,Казань,76117.0,119942.0
8,Краснодар,78483.0,117144.0
9,Самара,67885.0,101157.0


In [39]:
#Составлен запрос, позволяющий увидеть, как различаются среднее значение
#нижней и верхней границы в зависимости от требуемого опыта работы.
query_4_add_2 = f'''select 
                    experience,
                    round(avg(salary_from)) as avg_salary_min,
                    round(avg(salary_to)) as avg_salary_max
                from 
                    public.vacancies
                group by
                    experience
                order by 
                    avg_salary_max desc, avg_salary_min desc
'''

In [40]:
pd.read_sql_query(query_4_add_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\1095106340.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_add_2, connection)


,experience,avg_salary_min,avg_salary_max
0,Более 6 лет,150156.0,197819.0
1,От 3 до 6 лет,112544.0,171974.0
2,От 1 года до 3 лет,64456.0,97436.0
3,Нет опыта,34699.0,56990.0


In [43]:
#Составлен запрос, позволяющий увидеть, как различаются среднее значение
#нижней и верхней границы в зависимости от типа рабочего графика и типа трудоустройства.
query_4_add_3 = f'''select 
                    schedule,
                    employment,
                    round(avg(salary_from)) as avg_salary_min,
                    round(avg(salary_to)) as avg_salary_max
                from 
                    public.vacancies
                group by
                    schedule, employment
                order by count(*) desc
'''

In [44]:
pd.read_sql_query(query_4_add_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\2128982762.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_4_add_3, connection)


,schedule,employment,avg_salary_min,avg_salary_max
0,Полный день,Полная занятость,69165.0,101676.0
1,Удаленная работа,Полная занятость,100652.0,157314.0
2,Гибкий график,Полная занятость,94725.0,156357.0
3,Удаленная работа,Частичная занятость,22500.0,81262.0
4,Сменный график,Полная занятость,43502.0,51023.0
5,Полный день,Стажировка,29546.0,48637.0
6,Вахтовый метод,Полная занятость,106953.0,139904.0
7,Полный день,Частичная занятость,39507.0,69031.0
8,Гибкий график,Частичная занятость,37351.0,55199.0
9,Полный день,Проектная работа,77161.0,111779.0


In [23]:
#Более чем у половины вакансий отсутствует указание на потенциальный размер заработной платы.
#Для построения модели, прогнозирующей зарплату на основе имеющихся данных, необходимо
#в обязательном порядке провести дополнительную работу с пропусками.
#Подавляющее большинство вакансий в базе ориентированы на кандидатов, имеющих опыт работы.
#Вакансии, представленные в базе, распределены по различным регионам.
#Большая часть вакансий предполагае полную занятость на полный рабочий день.
#Также достаточно много вакансий для удаленной работы.
#На основании поверхностного анализа данных признаков можно предположить,
#что они влияют на различие в среднем уровне заработной платы.
#Соответственно после более детального изучения и прообработки
#данные признаки можно использовать для более качественного заполнения пустых полей с заработной платой.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [24]:
#Для составления запроса объединены таблица с работодателями присоединена к таблице с вакансиями.
#Данные сгруппированы по наименованию работодателя и отсортированы
#по количеству вакансий, размещенных работодателем. Оставлены первые пять значений.
n = 5
query_5_1 = f'''select 
                    count(v.id) as vacancies_count,
                    e.name as employers
                from 
                    public.vacancies as v
                join 
                    public.employers as e on v.employer_id = e.id
                group by 
                    e.name
                order by 
                    count(v.id) desc
                limit {n}
'''

In [25]:
pd.read_sql_query(query_5_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_1180\3902369145.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_1, connection)


,vacancies_count,employers
0,1933,Яндекс
1,491,Ростелеком
2,444,Тинькофф
3,428,СБЕР
4,331,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [26]:
#Для составления запроса объединены таблицы с вакансиями, работодателями присодинена к таблице с регионами.
#Данные сгруппированы по наименованию работодателя и отфильтрованы - оставлены данные
#о работодателях, кто разместил 0 вакансий. Данные отсортированы по количеству регионов
#присутствия таких работодателей.
query_5_2 = f'''select 
                    a.name as area,
                    count(v.id) as vacancies_count,
                    count(e.id) as employers_count
                from 
                    public.areas as a
                left join 
                    public.employers as e on a.id = e.area
                left join
                    public.vacancies as v on v.area_id = a.id
                group by 
                    a.name
                having
                    count(v.id) = 0
                order by 
                    count(e.id) desc
'''

In [27]:
pd.read_sql_query(query_5_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_1180\2146817738.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_2, connection)


,area,vacancies_count,employers_count
0,Россия,0,410
1,Казахстан,0,207
2,Московская область,0,75
3,Краснодарский край,0,19
4,Ростовская область,0,18
...,...,...,...
588,Струнино,0,0
589,Славгород (Алтайский край),0,0
590,Сорочинск,0,0
591,Ленинский (Тульская область),0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [29]:
#Для составления запроса объединены таблицы с регионами и работодателями присоединены к таблице с вакансиями.
#Данные сгруппированы по наименованию работодателя. Данные отсортированы по количеству уникальных регионов, где
#работодатель размещал вакансии.
query_5_3 = f'''select 
                    e.name as employer,
                    count(distinct a.id) as areas_count
                from
                    public.vacancies as v
                join 
                    public.employers as e on v.employer_id = e.id
                join
                    public.areas as a on v.area_id = a.id
                group by 
                    e.name
                order by 
                    count(distinct a.id) desc
'''

In [30]:
# результат запроса
pd.read_sql_query(query_5_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\593965358.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_3, connection)


,employer,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [30]:
#Таблица с индексами сфер деятельности присоединена к таблице с работодателями.
#Данные отфильтрованы на предмет того, указана ли сфера деятельности у конкретного работодателя.
query_5_4 = f'''select 
                    count (e.id) as employer_no_industry
                from
                    public.employers as e
                left join 
                    public.employers_industries as ei on ei.employer_id = e.id
                where
                    ei.industry_id is null
'''

In [31]:
pd.read_sql_query(query_5_4, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_1180\1644714913.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_4, connection)


,employer_no_industry
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [34]:
#Таблица с индексами сфер деятельности присоединена к таблице с работодателями.
#Данные сгруппированы по названию работодателя, сгруппированные данные отфильтрованы
#на предмет того, чтобы у работодателя было указано 4 сферы деятельности.
#Данные отфильтрованы в алфавитном порядке по наименованию работодателя.
query_5_5 = f'''select 
                    e.name,
                    count (ei.industry_id) as industries
                from
                    public.employers as e
                left join 
                    public.employers_industries as ei on ei.employer_id = e.id
                group by
                    e.name
                having
                    count (ei.industry_id) = 4
                order by
                    e.name
'''

In [35]:
pd.read_sql_query(query_5_5, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_1180\1169902027.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_5, connection)


,name,industries
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [31]:
#К таблице с названием сфер деятельности присоединена таблица с индексами работодателей.
#Данные сгруппированы по названию сферы деятельности.
#Данные отфильрованы на предмет того, чтобы в сфере деятельности была указана разработка ПО.
query_5_6 = f'''select 
                   i.name,
                   count (ei.employer_id) as employers_count
                from
                    public.industries as i
                left join 
                    public.employers_industries as ei on ei.industry_id = i.id
                group by
                    i.name
                having
                    i.name = 'Разработка программного обеспечения'
'''

In [32]:
pd.read_sql_query(query_5_6, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\2432745783.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_6, connection)


,name,employers_count
0,Разработка программного обеспечения,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [48]:
#С соответствующей страницы Википедии выгружены данные, содержащие наименования городов-миллионеров.
url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России#Список_городов-миллионеров'
content = requests.get(url)
soup = BeautifulSoup(content.text, 'html.parser')
table = soup.find('table', class_='standard sortable').find_all('a')[4:]

#Из выгруженных данных составлен список городов-миллионеров.
cities = []
for i in table:
    table = i.text
    cities.append(table)

#Список очищен от лишней информации и преобразован в кортеж.
cities.remove('[a]')
cities = tuple(cities)
    

In [63]:
#К таблице с вакансиями присоединены таблицы с работодателями и регионами.
#Данные сгруппированы по наименованию работодателя и региона.
#Данные отфильрованы на предмет того, чтобы в наименование работодателя
#был указан яндекс, а регион входил в список городов-миллионеров.
#Первый запрос объединен с другим запросом, отражащим общее количество вакансий
#Яндекса, размещенных в городах-миллионерах.
query_5_7 = f'''select 
                   a.name as area,
                   count (v.id) as vacancies
                from
                    public.vacancies as v
                join 
                    public.employers as e on v.employer_id = e.id
                join
                    public.areas as a on v.area_id = a.id
                group by
                    e.name, a.name
                having
                    e.name = 'Яндекс' and a.name in {cities}
                
                UNION ALL
                
                select 
                   'total',
                   count (v.id)
                from
                    public.vacancies as v
                join 
                    public.employers as e on v.employer_id = e.id
                join
                    public.areas as a on v.area_id = a.id
                where
                    a.name in {cities}
                group by
                    e.name
                having
                    e.name = 'Яндекс'
                order by
                    2
'''

In [64]:
pd.read_sql_query(query_5_7, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_6568\1216511710.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_7, connection)


,area,vacancies
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [60]:
#Составлен запрос, позволяющий понять, сколько ваканкий разместили работодатели,
#у которых в сфере деятельности указана разработка ПО.
#Для составления запрос к таблице с вакансиями присоединены таблицы с работодателями,
#индексом сфер деятельности, наименованием сфер деятельности.
#Данные отфильтрованы на предмет того, чтобы в сфере деятельности была указана разработка ПО,
#и сгруппированы по наименованию работодателя.
query_5_add_1 = f'''select 
                        e.name as employers,
                        count(v.id) as vacancies_count
                    from 
                        public.vacancies as v
                    join 
                        public.employers as e on v.employer_id = e.id
                    join 
                        public.employers_industries as ei on ei.employer_id = e.id
                    join
                        public.industries as i on ei.industry_id = i.id
                    where
                        i.name = 'Разработка программного обеспечения'
                    group by 
                        e.name
                    order by 
                        count(v.id) desc             
'''

In [59]:
pd.read_sql_query(query_5_add_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\2640994870.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_add_1, connection)


,employers,vacancies_count,areas
0,Тинькофф,444,43
1,DataArt,247,2
2,Совкомбанк Технологии,204,26
3,IBS,175,36
4,Тензор,155,25
...,...,...,...
2579,Индустрия делового программного обеспечения,1,1
2580,"Айтекс, Центр Информационных Технологий",1,1
2581,Айтек,1,1
2582,"ИНКОМ, группа компаний",1,1


In [37]:
#Составлен запрос, показывающий работодателей, кто не разместил ни одной вакансии.
#К таблице с работодателями присодинена таблица с вакансиями.
#Данные сгруппированы по наименования компании и отфилтрованы на предмет того, чтобы
#количество размещенных вакансий равнялось нулю.
query_5_add_2 = f'''select 
                        e.name
                    from 
                        public.employers as e
                    left join
                        public.vacancies as v on v.employer_id = e.id
                    group by 
                        e.id
                    having
                        count(v.id) = 0
'''

In [38]:
pd.read_sql_query(query_5_add_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\3456868961.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_5_add_2, connection)


,name
0,Северо-Западный Информационно-Мониторинговый Ц...
1,АВГ Инженерные Системы
2,КЕЛС
3,PR
4,ПКФ Станкор
...,...
8590,Винтаж
8591,Скайлинк
8592,Whitecliff Investment Management
8593,TransPerfect


In [ ]:
#Крупнейшим работодателем в базе данных является компания Яндекс.
#При этом около четверти вакансий Яндекса представлены в городах миллионниках,
#остальные 3/4 приходятся на 165 других регионов.
#Около трети работодателей не указали своей сферы деятельности, 
#разработка ПО указана в качестве сферы деятельности примерно у 15% работодателей.
#У многих работодателей указано несколько сфер деятельности.
#Однако многие крупнейшие работодатели из базы не указали разработки ПО в качестве сферы деятельности.
#В базе 8 595 работодателей и 593 региона без вакансий.
#Предлагается более детально проанализировать работодатели в каких сферах деятельности ищут специалистов на вакансию Data Scientist,
#по возможности заполнить сферу деятельности у работодателей, где она отсутствует.
#На основании проведенного анализа очистить данные от лишних работодателей и регионов.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [77]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data' или 'данн'.
query_6_1 = f'''select 
                   count(*) as data_vacancies
                from
                    public.vacancies
                where
                    lower(name) like '%data%' or lower(name) like '%данн%'
'''

In [78]:
pd.read_sql_query(query_6_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_6568\2603769340.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_1, connection)


,data_vacancies
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [112]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно данные отсортированы на предмет наличия в названии сочетания 'junior',
#отсуствия опыта в требуемом опыте или стажировки в типе трудоустройства, что говорит
#о том, что вакансия относится к уровню junior.
query_6_2 = f'''select 
                   count(*) as junior_data_vacancies
                from
                    public.vacancies
                where
                    (lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%')
                    and name not like '%HTML%'
                    and (lower(name) like '%junior%'
                    or experience = 'Нет опыта'
                    or employment = 'Стажировка')
'''

In [113]:
pd.read_sql_query(query_6_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_6568\2428688723.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_2, connection)


,junior_data_vacancies
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [73]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно данные отсортированы на предмет наличия в ключевых навыках сочетаний 'SQL' и 'postgres'.
query_6_3 = f'''select 
                   count(*) as sql_or_postgres
                from
                    public.vacancies
                where
                    (lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%')
                    and name not like '%HTML%'
                    and (key_skills like '%SQL%'
                    or key_skills like '%postgres%')
'''

In [74]:
pd.read_sql_query(query_6_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\3902879435.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_3, connection)


,sql_or_postgres
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [75]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно данные отсортированы на предмет наличия в ключевых навыках сочетания 'python'.
query_6_4 = f'''select 
                   count(*) as python_vacancies
                from
                    public.vacancies
                where
                    (lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%')
                    and name not like '%HTML%'
                    and lower(key_skills) like '%python%'
'''

In [76]:
pd.read_sql_query(query_6_4, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\2548700122.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_4, connection)


,python_vacancies
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [151]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно данные отсортированы на предмет отсутствия данных о ключевых навыках.
#Для вычисления среднего числа навыков мы рассчитали общую длинну строки с указанием навыков
#и вычли из нее длинну строки, в которой убрали разделение между навыками, и чтобы учесть
#первый навык в списке прибавили один. От полученного значения мы вычислили среднее.
query_6_5 = f'''select 
                  round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) +1), 2) as avg_skills_number
                from
                    public.vacancies
                where
                    (lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%')
                    and name not like '%HTML%'
                    and key_skills is not null
'''

In [152]:
pd.read_sql_query(query_6_5, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_6568\3854041181.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_5, connection)


,avg_skills_number
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [85]:
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно данные отсортированы на предмет отсутствия данных о вилке заработной платы.
#Для формирования данных о средней заработной плате мы сложили данные о нижней и верхней вилке зарплаты
#и поделили их на два. При этом отсутствующие данные в одном из полей с вилкой зарплаты
#мы заменили данными из второго поля с зарплатной вилкой. Далее мы вычислили среднее от полученного значения.
query_6_6 = f'''select 
                    experience,
                    round(avg((coalesce(salary_from, salary_to)+coalesce(salary_to, salary_from)) / 2)) as avg_salary
                from
                    public.vacancies
                where
                    (lower(name) like '%data scientist%'
                    or lower(name) like '%data science%'
                    or lower(name) like '%исследователь данных%'
                    or name like '%ML%'
                    or lower(name) like '%machine learning%'
                    or lower(name) like '%машинн%обучен%')
                    and name not like '%HTML%'
                    and (salary_from is not null
                    or salary_to is not null)
                group by
                    experience
'''

In [86]:
pd.read_sql_query(query_6_6, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\1796641676.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_6, connection)


,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

In [79]:
#Составлен запрос, показывающий среднее число навыков для всех вакансий в базе данных.
#Это позволит нам понять, насколько число навыков для вакансий data science соответствует среднему значению.
#Для вычисления среднего числа навыков мы рассчитали общую длинну строки с указанием навыков
#и вычли из нее длинну строки, в которой убрали разделение между навыками, и чтобы учесть
#первый навык в списке прибавили один. От полученного значения мы вычислили среднее.
query_6_add_1 = f'''select 
                      round(avg(length(key_skills) - length(replace(key_skills, CHR(9), '')) +1), 2) as avg_skills_number
                    from
                      public.vacancies
'''

In [80]:
pd.read_sql_query(query_6_add_1, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\2867181592.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_add_1, connection)


,avg_skills_number
0,6.37


In [95]:
#Составлен запрос, показывающий общую разбивку вакансий Data Science по требуемому опыту работу.
#Запрос позволит понять, какое количество вакансий с различным опытом, не содержат указаний на уровень зарплаты.
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно произведена филтрация на предмет отсутствия данных о зарплате. Данные сгруппированы по опыту работы.
query_6_add_2 = f'''select 
                        experience,
                        count(id)
                    from
                        public.vacancies
                    where
                        (lower(name) like '%data scientist%'
                        or lower(name) like '%data science%'
                        or lower(name) like '%исследователь данных%'
                        or name like '%ML%'
                        or lower(name) like '%machine learning%'
                        or lower(name) like '%машинн%обучен%')
                        and name not like '%HTML%'
                        and (salary_from is null
                        or salary_to is null)
                    group by
                        experience
'''

In [96]:
pd.read_sql_query(query_6_add_2, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_7144\3229580973.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_add_2, connection)


,experience,count
0,Более 6 лет,34
1,Нет опыта,25
2,От 1 года до 3 лет,209
3,От 3 до 6 лет,184


In [39]:
#Составлен запрос, показывающий общее количество вакансий Data Science, 
#у которых не заполнены данные о ключевых навыках.
#Запрос позволит понять, какое количество вакансий с различным опытом, не содержат указаний на уровень зарплаты.
#Данные по вакансиям отфильрованы на предмет наличия в их названиях
#сочетаний 'data scientist','data science','исследователь данных', 'ML',
#'machine learning','HTML', что свидетельствует о том, что они относятся к data science.
#Дополнительно произведена филтрация на предмет отсутствия данных о навыках.
query_6_add_3 = f'''select 
                        count(id) as vacancies_without_skills
                    from
                        public.vacancies
                    where
                        (lower(name) like '%data scientist%'
                        or lower(name) like '%data science%'
                        or lower(name) like '%исследователь данных%'
                        or name like '%ML%'
                        or lower(name) like '%machine learning%'
                        or lower(name) like '%машинн%обучен%')
                        and name not like '%HTML%'
                        and key_skills is null
'''

In [40]:
pd.read_sql_query(query_6_add_3, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\2005020700.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_add_3, connection)


,vacancies_without_skills
0,49


In [43]:
#Составлен запрос, показывающий общее количество вакансий Data Science, 
#у конкретных работодателей. Для составления запроса к таблице с вакансиями
#присоединена таблица с работодателями. Данные по вакансиям отфильрованы 
#на предмет наличия в их названиях сочетаний 'data scientist',
#'data science','исследователь данных', 'ML','machine learning','HTML', 
#что свидетельствует о том, что они относятся к data science.
#Данные сгруппированы по названию работодателей.
query_6_add_4 = f'''select 
                        e.name as employer,
                        count(v.id) as data_vacancies
                    from
                        public.vacancies as v
                    join
                        public.employers as e on e.id = v.employer_id
                    where
                        (lower(v.name) like '%data scientist%'
                        or lower(v.name) like '%data science%'
                        or lower(v.name) like '%исследователь данных%'
                        or v.name like '%ML%'
                        or lower(v.name) like '%machine learning%'
                        or lower(v.name) like '%машинн%обучен%')
                        and v.name not like '%HTML%'
                    group by
                        e.name
                    order by
                        count(v.id) desc
'''

In [44]:
pd.read_sql_query(query_6_add_4, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\2106468670.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_add_4, connection)


,employer,data_vacancies
0,СБЕР,37
1,Bell Integrator,25
2,Банк ВТБ (ПАО),18
3,VK,15
4,Positive Technologies,11
...,...,...
220,Иллекс,1
221,ИНВИТРО,1
222,Институт искусственного интеллекта AIRI,1
223,Интеллектуальные решения,1


In [45]:
#Составлен запрос, показывающий общее количество вакансий Data Science, 
#по конкретным регионам. Для составления запроса к таблице с вакансиями
#присоединена таблица с регионами. Данные по вакансиям отфильрованы 
#на предмет наличия в их названиях сочетаний 'data scientist',
#'data science','исследователь данных', 'ML','machine learning','HTML', 
#что свидетельствует о том, что они относятся к data science.
#Данные сгруппированы по названию региона.
query_6_add_5 = f'''select 
                        a.name as area,
                        count(v.id) as data_vacancies
                    from
                        public.vacancies as v
                    join
                        public.areas as a on a.id = v.area_id
                    where
                        (lower(v.name) like '%data scientist%'
                        or lower(v.name) like '%data science%'
                        or lower(v.name) like '%исследователь данных%'
                        or v.name like '%ML%'
                        or lower(v.name) like '%machine learning%'
                        or lower(v.name) like '%машинн%обучен%')
                        and v.name not like '%HTML%'
                    group by
                        a.name
                    order by
                        count(v.id) desc
'''

In [46]:
pd.read_sql_query(query_6_add_5, connection)

C:\Users\Люля\AppData\Local\Temp\ipykernel_260\4057533915.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql_query(query_6_add_5, connection)


,area,data_vacancies
0,Москва,217
1,Санкт-Петербург,64
2,Новосибирск,23
3,Нижний Новгород,20
4,Казань,16
5,Алматы,16
6,Минск,9
7,Томск,8
8,Екатеринбург,7
9,Краснодар,6


In [ ]:
#Около 3,5% от общего числа вакансий имеют отношение к Data Science.
#При этом вакансии уровня Junior составляют менее 0,5% от общего числа вакансий Data Science.
#Такие навыки как Python и SQL/postgres указаны в описании менее 3% и 2%
#от общего числа вакансий Data Science. При том, что среднее количество
#ключевых навыков для вакансий Data Science в базе данных составляет 6.41, что
#превышает аналогичный средний показатель для всех вакансий базы данных.
#Ключевые навыки не указаны для 0,5% вакансий Data Science.
#Расчитать средний показатель зарплаты возможно для небольшого числа вакансий,
#поскольку для большинства вакансий показатель зарплаты не указан, для опытных специалистов (более 6 лет)
#нет ни одной вакансии с показателям зарплаты.
#При этом у крупнейших работодателей из базы вакансий таких как Яндекс, Ростелеком, Тинькофф
#не представлено ни одной вакасий в области Data Science.
#При этом данные по вакансиями распределены по разлинчым регионам. 
#Большинство вакансий представлено в Москве и Санкт-Петербурге.

# Общий вывод по проекту

In [ ]:
#По итогам проведенного анализа имеющихся данных, а также дополнительного анализа,
#мне кажется, что имеющиеся данные не совсем отвечают поставленной задаче.
#База данных содержит много пропусков и лишней информации, не имеющей отношения к вакансиям Data Science.
#Кроме того, в базе данных представлено небольшое число вакансий Data Science.
#Имеющаяся информация в отношении данных вакансий неполная: отсутствуют данные по заработной плате,
#информация о ключевых навыках не всегда содержит навыки, характерные для Data Science.
#Прежде чем переходить к обучению модели рекомендуется провести очистку имеющихся данных.
#а также при наличии такой возможности провести работу с коллегами, ведущими базу данных, на предмет получения дополнительной
#информации по подобным вакансиям, а также дополнениями ее вакансиями от крупнейших работодателей.